In [1]:
import pandas as pd
import re
from nltk import BigramCollocationFinder
import nltk.collocations
import io
import os

In [2]:
df=pd.read_csv('/home/miki/Downloads/amharicdata/Scalable_Datawarehouse_Amharic_Data_Ingestion_For_LLM_RAG/data/raw/telegram_data.csv')

In [3]:
df.head(5)

,Channel,id,message,date
0,Eliasmeserett,323,💠እንኳን ለትንሳኤ በዓል አደረሳችሁ፣ አደረሰን! \n\nመልካም በዓል። \...,2024-05-04 14:10:54+00:00
1,Eliasmeserett,322,💠አስተያየት እና ጥቆማ ለምታደርሱኝ፣\n\nአካውንቶቼ ለተወሰኑ ሳምንታት ...,2024-04-16 04:38:46+00:00
2,Eliasmeserett,321,NaN,2024-04-16 04:10:01+00:00
3,Eliasmeserett,320,💠Genocide \n\nየዘር ፍጅት (ጄኖሳይድ) እንዳይፈጸም በመከላከል እ...,2024-04-16 04:10:01+00:00
4,Eliasmeserett,319,💠Update የአለም ምግብ ፕሮግራም (WFP) በድሬዳዋ ነፃ የንግድ ቀጠና...,2024-04-16 03:24:45+00:00


In [4]:
df.shape
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43753 entries, 0 to 43752
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Channel  43753 non-null  object
 1   id       43753 non-null  int64 
 2   message  27416 non-null  object
 3   date     43753 non-null  object
dtypes: int64(1), object(3)
memory usage: 1.3+ MB


In [5]:
missingCount = df.isnull().sum()
missingCount

Channel        0
id             0
message    16337
date           0
dtype: int64

In [6]:
df.dropna(subset=['message'], inplace=True)
df

,Channel,id,message,date
0,Eliasmeserett,323,💠እንኳን ለትንሳኤ በዓል አደረሳችሁ፣ አደረሰን! \n\nመልካም በዓል። \...,2024-05-04 14:10:54+00:00
1,Eliasmeserett,322,💠አስተያየት እና ጥቆማ ለምታደርሱኝ፣\n\nአካውንቶቼ ለተወሰኑ ሳምንታት ...,2024-04-16 04:38:46+00:00
3,Eliasmeserett,320,💠Genocide \n\nየዘር ፍጅት (ጄኖሳይድ) እንዳይፈጸም በመከላከል እ...,2024-04-16 04:10:01+00:00
4,Eliasmeserett,319,💠Update የአለም ምግብ ፕሮግራም (WFP) በድሬዳዋ ነፃ የንግድ ቀጠና...,2024-04-16 03:24:45+00:00
5,Eliasmeserett,318,👌👌\n\n@EliasMeserett,2024-04-15 05:01:11+00:00
...,...,...,...,...
43748,tikvahethsport,43336,ይደውሉልን 0911156257\nካሜራ እና የካሜራ እቃዎችን እንገዛለን እን...,2023-07-22 12:07:28+00:00
43749,tikvahethsport,43335,ከሶስት አስርት አመታት በኋላ የዛሬ አመት ኢትዮጵያ ግብፅን 2-0 ስታሸን...,2023-07-22 12:07:25+00:00
43750,tikvahethsport,43334,ሊቨርፑል በቀጣይ ተጨዋች ያስፈርማል ?\n\nየመርሲሳይዱ ክለብ ሊቨርፑል ...,2023-07-22 10:10:17+00:00
43751,tikvahethsport,43333,ሳሙኤል ኡምቲቲ ሊልን ተቀላቅሏል !\n\nፈረንሳዊው የተከላካይ ስፍራ ተጨ...,2023-07-22 08:41:10+00:00


In [7]:
def remove_ascii_and_numbers(text_input):
    rm_num_and_ascii=re.sub('[A-Za-z0-9]','',text_input)
    return re.sub('[\'\u1369-\u137C\']+','',rm_num_and_ascii)

In [8]:
df['cleaned_text'] = df['message'].apply(remove_ascii_and_numbers)
df

,Channel,id,message,date,cleaned_text
0,Eliasmeserett,323,💠እንኳን ለትንሳኤ በዓል አደረሳችሁ፣ አደረሰን! \n\nመልካም በዓል። \...,2024-05-04 14:10:54+00:00,💠እንኳን ለትንሳኤ በዓል አደረሳችሁ፣ አደረሰን! \n\nመልካም በዓል። \...
1,Eliasmeserett,322,💠አስተያየት እና ጥቆማ ለምታደርሱኝ፣\n\nአካውንቶቼ ለተወሰኑ ሳምንታት ...,2024-04-16 04:38:46+00:00,💠አስተያየት እና ጥቆማ ለምታደርሱኝ፣\n\nአካውንቶቼ ለተወሰኑ ሳምንታት ...
3,Eliasmeserett,320,💠Genocide \n\nየዘር ፍጅት (ጄኖሳይድ) እንዳይፈጸም በመከላከል እ...,2024-04-16 04:10:01+00:00,💠 \n\nየዘር ፍጅት (ጄኖሳይድ) እንዳይፈጸም በመከላከል እና ሲፈጸምም ...
4,Eliasmeserett,319,💠Update የአለም ምግብ ፕሮግራም (WFP) በድሬዳዋ ነፃ የንግድ ቀጠና...,2024-04-16 03:24:45+00:00,💠 የአለም ምግብ ፕሮግራም () በድሬዳዋ ነፃ የንግድ ቀጠና ዙርያ ባቀረብ...
5,Eliasmeserett,318,👌👌\n\n@EliasMeserett,2024-04-15 05:01:11+00:00,👌👌\n\n@
...,...,...,...,...,...
43748,tikvahethsport,43336,ይደውሉልን 0911156257\nካሜራ እና የካሜራ እቃዎችን እንገዛለን እን...,2023-07-22 12:07:28+00:00,ይደውሉልን \nካሜራ እና የካሜራ እቃዎችን እንገዛለን እንሸጣለን\n# \n...
43749,tikvahethsport,43335,ከሶስት አስርት አመታት በኋላ የዛሬ አመት ኢትዮጵያ ግብፅን 2-0 ስታሸን...,2023-07-22 12:07:25+00:00,ከሶስት አስርት አመታት በኋላ የዛሬ አመት ኢትዮጵያ ግብፅን - ስታሸንፍ ...
43750,tikvahethsport,43334,ሊቨርፑል በቀጣይ ተጨዋች ያስፈርማል ?\n\nየመርሲሳይዱ ክለብ ሊቨርፑል ...,2023-07-22 10:10:17+00:00,ሊቨርፑል በቀጣይ ተጨዋች ያስፈርማል ?\n\nየመርሲሳይዱ ክለብ ሊቨርፑል ...
43751,tikvahethsport,43333,ሳሙኤል ኡምቲቲ ሊልን ተቀላቅሏል !\n\nፈረንሳዊው የተከላካይ ስፍራ ተጨ...,2023-07-22 08:41:10+00:00,ሳሙኤል ኡምቲቲ ሊልን ተቀላቅሏል !\n\nፈረንሳዊው የተከላካይ ስፍራ ተጨ...


In [9]:
def remove_punc_and_special_chars(text):
    normalized_text = re.sub('[\!\@\#\$\%\^\«\»\&\*\(\)\…\[\]\{\}\;\“\”\›\’\‘\"\'\:\,\.\‹\/\<\>\?\\\\|\`\´\~\-\=\+\፡\።\፤\;\፦\፥\፧\፨\💠\👌\🙏\፠\፣]', '',text)
    return normalized_text

In [10]:
df['cleaned_text'] = df['message'].apply(remove_punc_and_special_chars)
df

,Channel,id,message,date,cleaned_text
0,Eliasmeserett,323,💠እንኳን ለትንሳኤ በዓል አደረሳችሁ፣ አደረሰን! \n\nመልካም በዓል። \...,2024-05-04 14:10:54+00:00,እንኳን ለትንሳኤ በዓል አደረሳችሁ አደረሰን \n\nመልካም በዓል \n\nE...
1,Eliasmeserett,322,💠አስተያየት እና ጥቆማ ለምታደርሱኝ፣\n\nአካውንቶቼ ለተወሰኑ ሳምንታት ...,2024-04-16 04:38:46+00:00,አስተያየት እና ጥቆማ ለምታደርሱኝ\n\nአካውንቶቼ ለተወሰኑ ሳምንታት ዝግ...
3,Eliasmeserett,320,💠Genocide \n\nየዘር ፍጅት (ጄኖሳይድ) እንዳይፈጸም በመከላከል እ...,2024-04-16 04:10:01+00:00,Genocide \n\nየዘር ፍጅት ጄኖሳይድ እንዳይፈጸም በመከላከል እና ሲ...
4,Eliasmeserett,319,💠Update የአለም ምግብ ፕሮግራም (WFP) በድሬዳዋ ነፃ የንግድ ቀጠና...,2024-04-16 03:24:45+00:00,Update የአለም ምግብ ፕሮግራም WFP በድሬዳዋ ነፃ የንግድ ቀጠና ዙር...
5,Eliasmeserett,318,👌👌\n\n@EliasMeserett,2024-04-15 05:01:11+00:00,\n\nEliasMeserett
...,...,...,...,...,...
43748,tikvahethsport,43336,ይደውሉልን 0911156257\nካሜራ እና የካሜራ እቃዎችን እንገዛለን እን...,2023-07-22 12:07:28+00:00,ይደውሉልን 0911156257\nካሜራ እና የካሜራ እቃዎችን እንገዛለን እን...
43749,tikvahethsport,43335,ከሶስት አስርት አመታት በኋላ የዛሬ አመት ኢትዮጵያ ግብፅን 2-0 ስታሸን...,2023-07-22 12:07:25+00:00,ከሶስት አስርት አመታት በኋላ የዛሬ አመት ኢትዮጵያ ግብፅን 20 ስታሸንፍ...
43750,tikvahethsport,43334,ሊቨርፑል በቀጣይ ተጨዋች ያስፈርማል ?\n\nየመርሲሳይዱ ክለብ ሊቨርፑል ...,2023-07-22 10:10:17+00:00,ሊቨርፑል በቀጣይ ተጨዋች ያስፈርማል \n\nየመርሲሳይዱ ክለብ ሊቨርፑል በ...
43751,tikvahethsport,43333,ሳሙኤል ኡምቲቲ ሊልን ተቀላቅሏል !\n\nፈረንሳዊው የተከላካይ ስፍራ ተጨ...,2023-07-22 08:41:10+00:00,ሳሙኤል ኡምቲቲ ሊልን ተቀላቅሏል \n\nፈረንሳዊው የተከላካይ ስፍራ ተጨዋ...


In [12]:
def normalize_char_level_missmatch(input_token):
        rep1=re.sub('[ሃኅኃሐሓኻ]','ሀ',input_token)
        rep2=re.sub('[ሑኁዅ]','ሁ',rep1)
        rep3=re.sub('[ኂሒኺ]','ሂ',rep2)
        rep4=re.sub('[ኌሔዄ]','ሄ',rep3)
        rep5=re.sub('[ሕኅ]','ህ',rep4)
        rep6=re.sub('[ኆሖኾ]','ሆ',rep5)
        rep7=re.sub('[ሠ]','ሰ',rep6)
        rep8=re.sub('[ሡ]','ሱ',rep7)
        rep9=re.sub('[ሢ]','ሲ',rep8)
        rep10=re.sub('[ሣ]','ሳ',rep9)
        rep11=re.sub('[ሤ]','ሴ',rep10)
        rep12=re.sub('[ሥ]','ስ',rep11)
        rep13=re.sub('[ሦ]','ሶ',rep12)
        rep14=re.sub('[ዓኣዐ]','አ',rep13)
        rep15=re.sub('[ዑ]','ኡ',rep14)
        rep16=re.sub('[ዒ]','ኢ',rep15)
        rep17=re.sub('[ዔ]','ኤ',rep16)
        rep18=re.sub('[ዕ]','እ',rep17)
        rep19=re.sub('[ዖ]','ኦ',rep18)
        rep20=re.sub('[ጸ]','ፀ',rep19)
        rep21=re.sub('[ጹ]','ፁ',rep20)
        rep22=re.sub('[ጺ]','ፂ',rep21)
        rep23=re.sub('[ጻ]','ፃ',rep22)
        rep24=re.sub('[ጼ]','ፄ',rep23)
        rep25=re.sub('[ጽ]','ፅ',rep24)
        rep26=re.sub('[ጾ]','ፆ',rep25)
        #Normalizing words with Labialized Amharic characters such as በልቱዋል or  በልቱአል to  በልቷል
        rep27=re.sub('(ሉ[ዋአ])','ሏ',rep26)
        rep28=re.sub('(ሙ[ዋአ])','ሟ',rep27)
        rep29=re.sub('(ቱ[ዋአ])','ቷ',rep28)
        rep30=re.sub('(ሩ[ዋአ])','ሯ',rep29)
        rep31=re.sub('(ሱ[ዋአ])','ሷ',rep30)
        rep32=re.sub('(ሹ[ዋአ])','ሿ',rep31)
        rep33=re.sub('(ቁ[ዋአ])','ቋ',rep32)
        rep34=re.sub('(ቡ[ዋአ])','ቧ',rep33)
        rep35=re.sub('(ቹ[ዋአ])','ቿ',rep34)
        rep36=re.sub('(ሁ[ዋአ])','ኋ',rep35)
        rep37=re.sub('(ኑ[ዋአ])','ኗ',rep36)
        rep38=re.sub('(ኙ[ዋአ])','ኟ',rep37)
        rep39=re.sub('(ኩ[ዋአ])','ኳ',rep38)
        rep40=re.sub('(ዙ[ዋአ])','ዟ',rep39)
        rep41=re.sub('(ጉ[ዋአ])','ጓ',rep40)
        rep42=re.sub('(ደ[ዋአ])','ዷ',rep41)
        rep43=re.sub('(ጡ[ዋአ])','ጧ',rep42)
        rep44=re.sub('(ጩ[ዋአ])','ጯ',rep43)
        rep45=re.sub('(ጹ[ዋአ])','ጿ',rep44)
        rep46=re.sub('(ፉ[ዋአ])','ፏ',rep45)
        rep47=re.sub('[ቊ]','ቁ',rep46) #ቁ can be written as ቊ
        rep48=re.sub('[ኵ]','ኩ',rep47) #ኩ can be also written as ኵ

        return rep48

In [13]:
df['cleaned_texts'] = df['cleaned_text'].apply(normalize_char_level_missmatch)
df

,Channel,id,message,date,cleaned_text,cleaned_texts
0,Eliasmeserett,323,💠እንኳን ለትንሳኤ በዓል አደረሳችሁ፣ አደረሰን! \n\nመልካም በዓል። \...,2024-05-04 14:10:54+00:00,እንኳን ለትንሳኤ በዓል አደረሳችሁ አደረሰን \n\nመልካም በዓል \n\nE...,እንኳን ለትንሳኤ በአል አደረሳችሁ አደረሰን \n\nመልካም በአል \n\nE...
1,Eliasmeserett,322,💠አስተያየት እና ጥቆማ ለምታደርሱኝ፣\n\nአካውንቶቼ ለተወሰኑ ሳምንታት ...,2024-04-16 04:38:46+00:00,አስተያየት እና ጥቆማ ለምታደርሱኝ\n\nአካውንቶቼ ለተወሰኑ ሳምንታት ዝግ...,አስተያየት እና ጥቆማ ለምታደርሱኝ\n\nአካውንቶቼ ለተወሰኑ ሳምንታት ዝግ...
3,Eliasmeserett,320,💠Genocide \n\nየዘር ፍጅት (ጄኖሳይድ) እንዳይፈጸም በመከላከል እ...,2024-04-16 04:10:01+00:00,Genocide \n\nየዘር ፍጅት ጄኖሳይድ እንዳይፈጸም በመከላከል እና ሲ...,Genocide \n\nየዘር ፍጅት ጄኖሳይድ እንዳይፈፀም በመከላከል እና ሲ...
4,Eliasmeserett,319,💠Update የአለም ምግብ ፕሮግራም (WFP) በድሬዳዋ ነፃ የንግድ ቀጠና...,2024-04-16 03:24:45+00:00,Update የአለም ምግብ ፕሮግራም WFP በድሬዳዋ ነፃ የንግድ ቀጠና ዙር...,Update የአለም ምግብ ፕሮግራም WFP በድሬዳዋ ነፃ የንግድ ቀጠና ዙር...
5,Eliasmeserett,318,👌👌\n\n@EliasMeserett,2024-04-15 05:01:11+00:00,\n\nEliasMeserett,\n\nEliasMeserett
...,...,...,...,...,...,...
43748,tikvahethsport,43336,ይደውሉልን 0911156257\nካሜራ እና የካሜራ እቃዎችን እንገዛለን እን...,2023-07-22 12:07:28+00:00,ይደውሉልን 0911156257\nካሜራ እና የካሜራ እቃዎችን እንገዛለን እን...,ይደውሉልን 0911156257\nካሜራ እና የካሜራ እቃዎችን እንገዛለን እን...
43749,tikvahethsport,43335,ከሶስት አስርት አመታት በኋላ የዛሬ አመት ኢትዮጵያ ግብፅን 2-0 ስታሸን...,2023-07-22 12:07:25+00:00,ከሶስት አስርት አመታት በኋላ የዛሬ አመት ኢትዮጵያ ግብፅን 20 ስታሸንፍ...,ከሶስት አስርት አመታት በኋላ የዛሬ አመት ኢትዮጵያ ግብፅን 20 ስታሸንፍ...
43750,tikvahethsport,43334,ሊቨርፑል በቀጣይ ተጨዋች ያስፈርማል ?\n\nየመርሲሳይዱ ክለብ ሊቨርፑል ...,2023-07-22 10:10:17+00:00,ሊቨርፑል በቀጣይ ተጨዋች ያስፈርማል \n\nየመርሲሳይዱ ክለብ ሊቨርፑል በ...,ሊቨርፑል በቀጣይ ተጨዋች ያስፈርማል \n\nየመርሲሳይዱ ክለብ ሊቨርፑል በ...
43751,tikvahethsport,43333,ሳሙኤል ኡምቲቲ ሊልን ተቀላቅሏል !\n\nፈረንሳዊው የተከላካይ ስፍራ ተጨ...,2023-07-22 08:41:10+00:00,ሳሙኤል ኡምቲቲ ሊልን ተቀላቅሏል \n\nፈረንሳዊው የተከላካይ ስፍራ ተጨዋ...,ሳሙኤል ኡምቲቲ ሊልን ተቀላቅሏል \n\nፈረንሳዊው የተከላካይ ስፍራ ተጨዋ...


In [22]:
class normalize(object):
    expansion_file_dir='../data/raw/telegram_data.csv' 
    # assume you have file with list of short forms with their expansion as gazeter
    short_form_dict={}
    # Constructor
    def __init__(self):
       self.short_form_dict=self.get_short_forms()

    def get_short_forms(self):
        text = open(self.expansion_file_dir, encoding='utf8')
        exp = {}
        for line in iter(text):
            line = line.strip()
            if not line:  # line is blank
                continue
            else:
                expanded = line.split("-")
                if len(expanded) == 2:  # Check if the line contains the hyphen separator
                    exp[expanded[0].strip()] = expanded[1].replace(" ", '_').strip()
                else:
                    print(f"Skipping line: {line} (does not contain the hyphen separator)")
        return exp

    # method that expand short form file
    def expand_short_form(self,input_short_word):
        if input_short_word in self.short_form_dict:
            return self.short_form_dict[input_short_word]
        else:
            return input_short_word

In [21]:
normalizer = normalize()

FileNotFoundError: [Errno 2] No such file or directory: './data/raw/telegram_data.csv'

In [32]:
df['cleaned_textss'] = df['cleaned_texts'].apply(normalizer.expand_short_form)
df.tail(50)


,Channel,id,message,date,cleaned_text,cleaned_texts,cleaned_textss
110437,EBCNEWSNOW,3545,“በአፍሪካ በኮቪድ-19 የሚያዙ ሰዎች ቁጥር እየቀነሰ ነው”፡- የዓለም ጤ...,2020-09-26 08:35:17+00:00,በአፍሪካ በኮቪድ19 የሚያዙ ሰዎች ቁጥር እየቀነሰ ነው የዓለም ጤና ድርጅ...,በአፍሪካ በኮቪድ19 የሚያዙ ሰዎች ቁጥር እየቀነሰ ነው የአለም ጤና ድርጅ...,በአፍሪካ በኮቪድ19 የሚያዙ ሰዎች ቁጥር እየቀነሰ ነው የአለም ጤና ድርጅ...
110438,EBCNEWSNOW,3544,የዘንድሮው የመስቀል ደመራ በዓል ውስን ሰዎች በተገኙበት ዛሬ ይከበራል\n...,2020-09-26 08:34:09+00:00,የዘንድሮው የመስቀል ደመራ በዓል ውስን ሰዎች በተገኙበት ዛሬ ይከበራል\n...,የዘንድሮው የመስቀል ደመራ በአል ውስን ሰዎች በተገኙበት ዛሬ ይከበራል\n...,የዘንድሮው የመስቀል ደመራ በአል ውስን ሰዎች በተገኙበት ዛሬ ይከበራል\n...
110439,EBCNEWSNOW,3543,ህብረተሰቡ የመስቀል በዓልን ሲያከብር ለኮሮናቫይረስ አጋላጭ ከሆኑ ድርጊቶ...,2020-09-26 08:33:32+00:00,ህብረተሰቡ የመስቀል በዓልን ሲያከብር ለኮሮናቫይረስ አጋላጭ ከሆኑ ድርጊቶ...,ህብረተሰቡ የመስቀል በአልን ሲያከብር ለኮሮናቫይረስ አጋላጭ ከሆኑ ድርጊቶ...,ህብረተሰቡ የመስቀል በአልን ሲያከብር ለኮሮናቫይረስ አጋላጭ ከሆኑ ድርጊቶ...
110440,EBCNEWSNOW,3542,ቀዳማዊት እመቤት ዝናሽ ታያቸው ለ72 አካል ጉዳተኞች የዊልቼር ድጋፍ አበ...,2020-09-26 08:32:58+00:00,ቀዳማዊት እመቤት ዝናሽ ታያቸው ለ72 አካል ጉዳተኞች የዊልቼር ድጋፍ አበ...,ቀዳማዊት እመቤት ዝናሽ ታያቸው ለ72 አካል ጉዳተኞች የዊልቼር ድጋፍ አበ...,ቀዳማዊት እመቤት ዝናሽ ታያቸው ለ72 አካል ጉዳተኞች የዊልቼር ድጋፍ አበ...
110441,EBCNEWSNOW,3541,በኢትዮጵያ 486 ተጨማሪ ሰዎች ኮሮናቫይረስ ሲገኝባቸው፤ 402 ሰዎች ደግ...,2020-09-25 16:38:30+00:00,በኢትዮጵያ 486 ተጨማሪ ሰዎች ኮሮናቫይረስ ሲገኝባቸው 402 ሰዎች ደግሞ...,በኢትዮጵያ 486 ተጨማሪ ሰዎች ኮሮናቫይረስ ሲገኝባቸው 402 ሰዎች ደግሞ...,በኢትዮጵያ 486 ተጨማሪ ሰዎች ኮሮናቫይረስ ሲገኝባቸው 402 ሰዎች ደግሞ...
110442,EBCNEWSNOW,3540,የሰላም ግንባታ ስራዎች ህዝብን ማዕከል ማድረግ እንዳለባቸው ተገለፀ\n**...,2020-09-25 13:50:39+00:00,የሰላም ግንባታ ስራዎች ህዝብን ማዕከል ማድረግ እንዳለባቸው ተገለፀ\n\n...,የሰላም ግንባታ ስራዎች ህዝብን ማእከል ማድረግ እንዳለባቸው ተገለፀ\n\n...,የሰላም ግንባታ ስራዎች ህዝብን ማእከል ማድረግ እንዳለባቸው ተገለፀ\n\n...
110443,EBCNEWSNOW,3539,ተፈናቃዮችን በዘላቂነት ለማቋቋም ተቀናጅቶ መሥራት እንዳለበት የጤና ሚኒስ...,2020-09-25 13:32:14+00:00,ተፈናቃዮችን በዘላቂነት ለማቋቋም ተቀናጅቶ መሥራት እንዳለበት የጤና ሚኒስ...,ተፈናቃዮችን በዘላቂነት ለማቋቋም ተቀናጅቶ መስራት እንዳለበት የጤና ሚኒስ...,ተፈናቃዮችን በዘላቂነት ለማቋቋም ተቀናጅቶ መስራት እንዳለበት የጤና ሚኒስ...
110444,EBCNEWSNOW,3538,የአዲስ አበባ ከተማ አስተዳደር ለጤና ባለሙያዎች የገቢ ግብር ወጪያቸውን ...,2020-09-25 13:08:02+00:00,የአዲስ አበባ ከተማ አስተዳደር ለጤና ባለሙያዎች የገቢ ግብር ወጪያቸውን ...,የአዲስ አበባ ከተማ አስተዳደር ለጤና ባለሙያዎች የገቢ ግብር ወጪያቸውን ...,የአዲስ አበባ ከተማ አስተዳደር ለጤና ባለሙያዎች የገቢ ግብር ወጪያቸውን ...
110445,EBCNEWSNOW,3537,ከማንነትና ከራስ አስተዳደር ጋር የተያያዙ ችግሮችን ለመፍታት የሚያስችል ...,2020-09-25 12:43:50+00:00,ከማንነትና ከራስ አስተዳደር ጋር የተያያዙ ችግሮችን ለመፍታት የሚያስችል ...,ከማንነትና ከራስ አስተዳደር ጋር የተያያዙ ችግሮችን ለመፍታት የሚያስችል ...,ከማንነትና ከራስ አስተዳደር ጋር የተያያዙ ችግሮችን ለመፍታት የሚያስችል ...
110446,EBCNEWSNOW,3536,በመተከል ዞን ቡለን ወረዳ ተከስቶ በነበረው ግጭት ለተፈናቀሉ ዜጎች የሚው...,2020-09-25 11:15:52+00:00,በመተከል ዞን ቡለን ወረዳ ተከስቶ በነበረው ግጭት ለተፈናቀሉ ዜጎች የሚው...,በመተከል ዞን ቡለን ወረዳ ተከስቶ በነበረው ግጭት ለተፈናቀሉ ዜጎች የሚው...,በመተከል ዞን ቡለን ወረዳ ተከስቶ በነበረው ግጭት ለተፈናቀሉ ዜጎች የሚው...


In [ ]:
clean_df=df['id','channel', 'cleaned_textss','date']
clean_df